In [2]:
import logging

from logger_config import setup_logging

setup_logging(level=logging.INFO)


import configparser

import pandas as pd
import main

from economic_data.extract.economic_data import (
    fetch_ecb_json,
    fetch_eurostat_json,
    fetch_fred_json,
)
from economic_data.transform.transform_economic_data import (
    calculate_monthly_change,
    ecb_json_to_df,
    eurostat_json_to_df,
    fred_json_to_df,
    label_and_append,
    set_monthly_ecb_interest_rate,
    rename_economic_indicators,
    threshold_csv_to_df,
)

logger = logging.getLogger(__name__)
pd.set_option("display.max_rows", 500)

In [3]:
file = "config/economic_thresholds_europe_us.csv"

df_thresholds = pd.read_csv(file)
df_thresholds

,indicator,good_range,medium_range,bad_range
0,inflation_monthly_euro,0.2% – 0.4%,0.0% – 0.2% or 0.4% – 0.8%,< 0% or > 0.8%
1,interest_rate_monthly_euro,0.5% – 2.5%,0.0% – 0.5% or 2.5% – 4.0%,< 0.0% or > 4.0%
2,unemployment_rate_monthly_euro,5.5% – 7.0%,4.5% – 5.5% or 7.0% – 9.0%,< 4.5% or > 9.0%
3,inflation_monthly_us,0.2% – 0.4%,0.0% – 0.2% or 0.4% – 0.8%,< 0% or > 0.8%
4,interest_rate_monthly_us,1.5% – 3.5%,0.5% – 1.5% or 3.5% – 5.5%,< 0.5% or > 5.5%
5,unemployment_monthly_rate_us,3.5% – 5.0%,2.5% – 3.5% or 5.0% – 7.0%,< 2.5% or > 7.0%


In [4]:
df = main.main()
df

2025-05-31 07:37:20,777 - INFO - main - Starting economic data extraction and transformation...
2025-05-31 07:37:20,778 - INFO - fetch_eurostat_json - Fetching Eurostat data for prc_hicp_mmor and from 2017-01
2025-05-31 07:37:22,498 - INFO - fetch_eurostat_json - Fetching Eurostat data for ei_lmhr_m and from 2017-01
2025-05-31 07:37:22,674 - INFO - fetch_ecb_json - Fetching ECB data for dataflow FM, series B.U2.EUR.4F.KR.MRR_FR.LEV, from 2017-01 to 2025-05
2025-05-31 07:37:23,026 - INFO - fetch_fred_json - Fetching FRED data for series ID UNRATE, from 2017-01-01 to 
2025-05-31 07:37:23,233 - INFO - fetch_fred_json - Fetching FRED data for series ID CPIAUCSL, from 2017-01-01 to 
2025-05-31 07:37:23,481 - INFO - fetch_fred_json - Fetching FRED data for series ID DFF, from 2017-01-01 to 
2025-05-31 07:37:23,754 - INFO - eurostat_json_to_df - Transformed Eurostat data for prc_hicp_mmor: 100 records
2025-05-31 07:37:23,756 - INFO - eurostat_json_to_df - Transformed Eurostat data for ei_lmhr

,date,value,indicator,source,unit,score
0,2017-01-01,-0.700000,inflation_monthly_euro,Eurostat,Percent,0.0
1,2017-02-01,0.400000,inflation_monthly_euro,Eurostat,Percent,2.0
2,2017-03-01,0.700000,inflation_monthly_euro,Eurostat,Percent,1.0
3,2017-04-01,0.500000,inflation_monthly_euro,Eurostat,Percent,1.0
4,2017-05-01,-0.100000,inflation_monthly_euro,Eurostat,Percent,0.0
...,...,...,...,...,...,...
713,2024-12-01,0.364672,inflation_monthly_us,FRED,Percent,2.0
714,2025-01-01,0.466935,inflation_monthly_us,FRED,Percent,1.0
715,2025-02-01,0.215929,inflation_monthly_us,FRED,Percent,2.0
716,2025-03-01,-0.050035,inflation_monthly_us,FRED,Percent,0.0


In [ ]:
df.sort_values(by="date").head(100)

,date,value,indicator,source,unit,score
0,2017-01-01,-0.700000,inflation_monthly_euro,Eurostat,Percent,0.0
217,2017-01-01,0.000000,interest_rate_monthly_euro,ECB,Percent per annum,1.0
418,2017-01-01,243.618000,inflation_index_monthly_us,FRED,Index,NaN
518,2017-01-01,0.650000,interest_rate_monthly_us,FRED,Percent,1.0
100,2017-01-01,9.100000,unemployment_rate_monthly_euro,Eurostat,Percent,0.0
318,2017-01-01,4.700000,unemployment_monthly_rate_us,FRED,Percent,2.0
101,2017-02-01,9.100000,unemployment_rate_monthly_euro,Eurostat,Percent,0.0
519,2017-02-01,0.660000,interest_rate_monthly_us,FRED,Percent,1.0
319,2017-02-01,4.600000,unemployment_monthly_rate_us,FRED,Percent,2.0
619,2017-02-01,0.159266,inflation_monthly_us,FRED,Percent,1.0
